# PyDipAPI - Content Parser Tutorial

Dieses Notebook demonstriert die Content-Parser Features von PyDipAPI.

## Was Sie lernen werden:
1. **ProtocolParser** - Plenarprotokolle analysieren
2. **DocumentParser** - Drucksachen verarbeiten
3. **PersonParser** - Abgeordneten-Daten extrahieren
4. **ActivityParser** - Aktivitaeten analysieren
5. **Batch-Parsing** - Performance-Optimierung
6. **Praktische Use Cases** - Reale Anwendungsfaelle
7. **Uebungsaufgaben** - Ihr Wissen testen

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

from pydipapi import DipAnfrage
from pydipapi.parsers import ProtocolParser, DocumentParser, PersonParser, ActivityParser

# API-Schluessel konfigurieren
API_KEY = "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN"

if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    dip = DipAnfrage(api_key=API_KEY)
    print("Content Parser Tutorial bereit!")
else:
    print("BITTE ERSETZEN SIE DEN PLATZHALTER MIT IHREM ECHTEN API-SCHLUESSEL!")
    print("Den API-Schluessel erhalten Sie unter: https://dip.bundestag.de/api/")

## ProtocolParser - Plenarprotokolle analysieren

Der ProtocolParser extrahiert strukturierte Informationen aus Plenarprotokollen.

In [ ]:
# Plenarprotokoll laden und parsen
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    try:
        print("Teste ProtocolParser...")
        protocols = dip.get_plenarprotokoll(anzahl=1)
        if protocols:
            parser = ProtocolParser()
            parsed = parser.parse(protocols[0])
            print(f"Protokoll geparst: {len(parsed)} Felder")
            print(f"Verfuegbare Felder: {list(parsed.keys())[:10]}...")
            
            # Beispiel-Felder anzeigen
            if 'datum' in parsed:
                print(f"Datum: {parsed.get('datum')}")
            if 'wahlperiode' in parsed:
                print(f"Wahlperiode: {parsed.get('wahlperiode')}")
        else:
            print("Keine Protokolle gefunden")
    except Exception as e:
        print(f"Fehler: {e}")
else:
    print("Bitte API-Schluessel konfigurieren.")

## DocumentParser - Drucksachen analysieren

Der DocumentParser strukturiert Drucksachen-Daten fuer einfachere Analyse.

In [ ]:
# Drucksachen laden und parsen
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    try:
        print("Teste DocumentParser...")
        documents = dip.get_drucksache(anzahl=3)
        if documents:
            parser = DocumentParser()
            
            # Einzelnes Dokument parsen
            parsed_doc = parser.parse(documents[0])
            print(f"Dokument geparst: {len(parsed_doc)} Felder")
            print(f"Beispiel-Felder: {list(parsed_doc.keys())[:5]}")
            
            # Batch-Parsing
            print("\nBatch-Parsing...")
            batch_results = parser.parse_batch(documents)
            print(f"Batch-Parsing: {len(batch_results)} Dokumente erfolgreich geparst")
            
            # Beispiel: Dokumenttypen extrahieren
            doc_types = [doc.get('drucksachetyp', 'Unbekannt') for doc in batch_results]
            print(f"\nDokumenttypen: {set(doc_types)}")
        else:
            print("Keine Dokumente gefunden")
    except Exception as e:
        print(f"Fehler: {e}")
else:
    print("Bitte API-Schluessel konfigurieren.")

## PersonParser - Abgeordnete analysieren

Der PersonParser extrahiert und strukturiert Abgeordneten-Daten.

In [ ]:
# Personen laden und parsen
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    try:
        print("Teste PersonParser...")
        persons = dip.get_person(anzahl=5)
        if persons:
            parser = PersonParser()
            batch_persons = parser.parse_batch(persons)
            
            print(f"{len(batch_persons)} Personen geparst")
            
            # Beispiel-Person anzeigen
            if batch_persons:
                person = batch_persons[0]
                print(f"\nBeispiel-Person:")
                print(f"  Name: {person.get('name', 'Unbekannt')}")
                print(f"  Fraktion: {person.get('fraktion', 'Unbekannt')}")
                print(f"  Partei: {person.get('partei', 'Unbekannt')}")
            
            # Fraktionen analysieren
            from collections import Counter
            fraktionen = [p.get('fraktion', 'Unbekannt') for p in batch_persons]
            fraktion_counts = Counter(fraktionen)
            print(f"\nFraktionen-Verteilung:")
            for fraktion, count in fraktion_counts.most_common():
                print(f"  {fraktion}: {count}")
        else:
            print("Keine Personen gefunden")
    except Exception as e:
        print(f"Fehler: {e}")
else:
    print("Bitte API-Schluessel konfigurieren.")

## ActivityParser - Aktivitaeten analysieren

Der ActivityParser strukturiert parlamentarische Aktivitaeten.

In [ ]:
# Aktivitaeten laden und parsen
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    try:
        print("Teste ActivityParser...")
        activities = dip.get_aktivitaet(anzahl=10)
        if activities:
            parser = ActivityParser()
            batch_activities = parser.parse_batch(activities)
            
            print(f"{len(batch_activities)} Aktivitaeten geparst")
            
            # Aktivitaets-Typen analysieren
            from collections import Counter
            types = [a.get('aktivitaetstyp', 'Unbekannt') for a in batch_activities]
            type_counts = Counter(types)
            
            print("\nAktivitaets-Typen:")
            for activity_type, count in type_counts.most_common():
                print(f"  - {activity_type}: {count}")
        else:
            print("Keine Aktivitaeten gefunden")
    except Exception as e:
        print(f"Fehler: {e}")
else:
    print("Bitte API-Schluessel konfigurieren.")

## Praktische Use Cases

### Use Case 1: Analyse von Drucksachen nach Typ

Analysieren Sie die Verteilung von Drucksachen-Typen in einer bestimmten Wahlperiode.

In [ ]:
# Use Case: Drucksachen-Typen analysieren
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    try:
        print("Analysiere Drucksachen-Typen der 20. Wahlperiode...")
        
        # Drucksachen abrufen
        documents = dip.get_drucksache(anzahl=20, wahlperiode=20)
        
        if documents:
            parser = DocumentParser()
            parsed_docs = parser.parse_batch(documents)
            
            # Typen extrahieren und zaehlen
            from collections import Counter
            doc_types = [doc.get('drucksachetyp', 'Unbekannt') for doc in parsed_docs]
            type_counts = Counter(doc_types)
            
            print(f"\nGefundene Drucksachen-Typen:")
            for doc_type, count in type_counts.most_common():
                percentage = (count / len(parsed_docs)) * 100
                print(f"  {doc_type}: {count} ({percentage:.1f}%)")
        else:
            print("Keine Dokumente gefunden")
    except Exception as e:
        print(f"Fehler: {e}")
else:
    print("Bitte API-Schluessel konfigurieren.")

### Use Case 2: Abgeordnete nach Fraktion gruppieren

Gruppieren Sie Abgeordnete nach ihrer Fraktion und analysieren Sie die Verteilung.

In [ ]:
# Use Case: Abgeordnete nach Fraktion gruppieren
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    try:
        print("Analysiere Abgeordnete nach Fraktion...")
        
        # Personen abrufen
        persons = dip.get_person(anzahl=30)
        
        if persons:
            parser = PersonParser()
            parsed_persons = parser.parse_batch(persons)
            
            # Nach Fraktion gruppieren
            from collections import defaultdict
            fraktion_groups = defaultdict(list)
            
            for person in parsed_persons:
                fraktion = person.get('fraktion', 'Unbekannt')
                name = f"{person.get('vorname', '')} {person.get('nachname', '')}".strip()
                if name:
                    fraktion_groups[fraktion].append(name)
            
            print(f"\nFraktionen-Verteilung:")
            for fraktion, members in sorted(fraktion_groups.items(), key=lambda x: len(x[1]), reverse=True):
                print(f"\n{fraktion}: {len(members)} Mitglieder")
                if len(members) <= 5:
                    for member in members:
                        print(f"  - {member}")
        else:
            print("Keine Personen gefunden")
    except Exception as e:
        print(f"Fehler: {e}")
else:
    print("Bitte API-Schluessel konfigurieren.")

### Use Case 3: Aktivitaeten-Trends analysieren

Analysieren Sie die Verteilung von Aktivitaetstypen ueber einen Zeitraum.

In [ ]:
# Use Case: Aktivitaeten-Trends
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    try:
        from datetime import datetime, timedelta
        
        print("Analysiere Aktivitaeten der letzten 30 Tage...")
        
        # Datum berechnen
        date_from = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
        
        # Aktivitaeten abrufen
        activities = dip.get_aktivitaet(anzahl=50, datum_von=date_from)
        
        if activities:
            parser = ActivityParser()
            parsed_activities = parser.parse_batch(activities)
            
            # Aktivitaetstypen analysieren
            from collections import Counter
            types = [a.get('aktivitaetstyp', 'Unbekannt') for a in parsed_activities]
            type_counts = Counter(types)
            
            print(f"\nAktivitaeten-Trends (letzte 30 Tage):")
            print(f"Gesamt: {len(parsed_activities)} Aktivitaeten")
            print(f"\nTop Aktivitaetstypen:")
            for activity_type, count in type_counts.most_common(5):
                percentage = (count / len(parsed_activities)) * 100
                print(f"  {activity_type}: {count} ({percentage:.1f}%)")
        else:
            print("Keine Aktivitaeten gefunden")
    except Exception as e:
        print(f"Fehler: {e}")
else:
    print("Bitte API-Schluessel konfigurieren.")

## Uebungsaufgaben

Testen Sie Ihr Wissen mit diesen praktischen Aufgaben:

### Aufgabe 1: Dokumenttypen zaehlen

Schreiben Sie Code, der:
1. 20 Drucksachen der 20. Wahlperiode abruft
2. Diese mit DocumentParser parst
3. Die haeufigsten 3 Dokumenttypen ausgibt

In [ ]:
# Ihre Loesung hier:
# Tipp: Verwenden Sie DocumentParser, Counter und most_common()


### Aufgabe 2: Personen nach Partei filtern

Schreiben Sie Code, der:
1. 30 Personen abruft
2. Diese mit PersonParser parst
3. Alle Personen einer bestimmten Partei/Fraktion findet und ausgibt

In [ ]:
# Ihre Loesung hier:
# Tipp: Verwenden Sie PersonParser und eine Liste mit Filter


### Aufgabe 3: Aktivitaeten-Statistik

Schreiben Sie Code, der:
1. Die letzten 20 Aktivitaeten abruft
2. Diese mit ActivityParser parst
3. Eine Statistik erstellt: Anzahl pro Aktivitaetstyp und Durchschnitt pro Tag

In [ ]:
# Ihre Loesung hier:
# Tipp: Verwenden Sie ActivityParser, Counter und datetime


## Zusammenfassung

In diesem Notebook haben Sie gelernt:

- **ProtocolParser**: Plenarprotokolle strukturiert analysieren
- **DocumentParser**: Drucksachen und Dokumente verarbeiten
- **PersonParser**: Abgeordneten-Daten extrahieren
- **ActivityParser**: Aktivitaeten und Abstimmungen analysieren
- **Batch-Parsing**: Effiziente Verarbeitung mehrerer Datensaetze
- **Praktische Use Cases**: Reale Anwendungsfaelle
- **Uebungsaufgaben**: Ihr Wissen testen

### Naechste Schritte

Weiter mit **Notebook 05**: Async API fuer maximale Performance!